In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from tqdm import tqdm_notebook
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier

tqdm.pandas()

In [3]:
structure = pd.read_csv("kaggle/structure.csv")
targets = pd.read_csv("kaggle/targets.csv")
df = pd.read_csv("kaggle/user_activity.csv")

In [4]:
costed_steps = structure[structure.step_cost > 0]

In [9]:
df[(df.step_id.isin(costed_steps.step_id)) & (df.user_id == 16953)]

,user_id,action,step_id,time,step_type,step_cost
154347,16953,discovered,4289,1410894805,choice,1
154348,16953,viewed,4289,1410894805,choice,1
154355,16953,started_attempt,4289,1410894809,choice,1
361732,16953,discovered,2918,1411490384,code,2
361733,16953,viewed,2918,1411490384,code,2
361737,16953,discovered,2919,1411490389,code,2
361738,16953,viewed,2919,1411490389,code,2
361740,16953,discovered,4969,1411490391,code,2
361741,16953,viewed,4969,1411490391,code,2
361760,16953,discovered,7422,1411490400,code,2


In [215]:
def f(x):
    passed = x[x.action == "passed"]
    not_passed = x[~(x.step_id.isin(passed.step_id))]
    
    avg_attempts = np.mean(not_passed[not_passed.action == "started_attempt"].time.values)
    
    not_passed = not_passed.drop_duplicates("step_id")
#     print(not_passed)
    days = 10*3600*24
    
    border = passed.time.max() - days
    
    passed_steps = len([1 for t in passed.time if t >= border])
    skipped_steps = len([1 for t in not_passed.time if t >= border])
    
    t = []
    for step_id in passed.step_id:
        tx = x[x.step_id == step_id].time
        t.append(tx.max() - tx.min())
    if t:
        avg_time = np.mean(t)
    else:
        avg_time = 0
    
    if passed_steps + skipped_steps + avg_time:
        return [passed_steps, skipped_steps, avg_time]

# TODO: Number of skipped steps and substraction of passed and skipped

In [216]:
x = df[df.step_id.isin(costed_steps.step_id)].groupby("user_id").get_group(16953)

In [217]:
f(x)

[10, 4, 107307.7]

In [218]:
X_t = df[df.step_id.isin(costed_steps.step_id)].groupby("user_id").progress_apply(f)
Y_t = targets[targets.user_id.isin(X_t.index)]


 48%|████▊     | 2060/4256 [07:20<07:49,  4.67it/s]

100%|██████████| 9426/9426 [01:35<00:00, 98.97it/s] 

In [223]:
X_t.values, Y_t.passed.values

(array([[9, 0, 632.44444444444446], [9, 0, 15078.222222222223],
        [20, 0, 9834.7000000000007], ..., [4, 1, 60.0], [2, 1, 100.5],
        [3, 0, 144.0]], dtype=object), array([0, 0, 0, ..., 0, 0, 0]))

In [224]:
X = np.array([np.array(x) for x in X_t.values]).reshape(-1, 3)
Y = Y_t.passed.ravel()

In [225]:
X

array([[  9.00000000e+00,   0.00000000e+00,   6.32444444e+02],
       [  9.00000000e+00,   0.00000000e+00,   1.50782222e+04],
       [  2.00000000e+01,   0.00000000e+00,   9.83470000e+03],
       ..., 
       [  4.00000000e+00,   1.00000000e+00,   6.00000000e+01],
       [  2.00000000e+00,   1.00000000e+00,   1.00500000e+02],
       [  3.00000000e+00,   0.00000000e+00,   1.44000000e+02]])

In [226]:
classifier = SGDClassifier()
classifier.coef_ = [2.47667795, -2.75186439]
classifier.intercept_ = [-25.97229747]

In [227]:
best_params = [None, None]
scores = []
best_score = 0
for _ in tqdm_notebook(range(500)):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
    classifier.fit(X_train, Y_train)
    Y_pred = classifier.predict(X_test)
    if Y_pred.sum() > 0:
        score = f1_score(Y_test, Y_pred)
        if score > best_score:
            print(Y_pred.sum())
            best_score = max(best_score, score)
            best_params = [classifier.coef_, classifier.intercept_] 
        scores.append(score)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

1904


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

...

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [228]:
np.mean(scores)

0.13668172440530679

In [229]:
best_params

[array([[   205.35200785,  -1285.22615785,  74013.38980988]]),
 array([-2855.638779])]

In [197]:
def prepare(df, num_features):
    X = df.groupby("user_id").progress_apply(f)
    return X.index, np.array([np.array(x) for x in X.values]).reshape(-1, num_features)

In [230]:
def f(x):
    passed = x[x.action == "passed"]
    not_passed = x[~(x.step_id.isin(passed.step_id))]
    not_passed = not_passed.drop_duplicates("step_id")
#     print(not_passed)
    days = 10*3600*24
    
    border = passed.time.max() - days
    
    passed_steps = len([1 for t in passed.time if t >= border])
    skipped_steps = len([1 for t in not_passed.time if t >= border])
    
    t = []
    for step_id in passed.step_id:
        tx = x[x.step_id == step_id].time
        t.append(tx.max() - tx.min())
    if t:
        avg_time = np.mean(t)
    else:
        avg_time = 0
    return [passed_steps, skipped_steps, avg_time]

# TODO: Number of skipped steps and substraction of passed and skipped

In [231]:
user_ids, X_val = prepare(pd.read_csv("kaggle/user_activity_test.csv"), 3)

100%|██████████| 4256/4256 [01:59<00:00, 35.71it/s]


In [232]:
X_train[5], X_val[5]

(array([  8.00000000e+00,   3.00000000e+00,   1.19784118e+04]),
 array([ 30.        ,   0.        ,  97.66666667]))

In [233]:
predicted = classifier.predict(X_val)

In [234]:
res = pd.DataFrame({"user_id": user_ids, "passed": predicted}).set_index("user_id")

In [235]:
res.passed.value_counts()

0    3874
1     381
Name: passed, dtype: int64

In [236]:
res.to_csv("result.csv")